# PDG API tutorial, PyHEP 2024

## Introduction

This is a whirlwind tour of the new PDG Python API!

## Installation

(Optional) First, create a virtual environment and activate it:

```bash
python -m venv ~/pdg.venv
source ~/pdg.venv/bin/activate
```

Now install the package:

```bash
pip install pdg
```

## Preamble

Unless otherwise noted, we assume the following preamble has been run:

In [1]:
import pdg
from pdg.particle import PdgParticle

api = pdg.connect()

## Concepts 

### Editions

There are multiple editions of the PDG database corresponding to the (bi)annual PDG releases. There is also a pdgall.sqlite containing data from all historical editions.

### PDG IDs and nodes 

Here, *PDG IDs* are used to to identify particles, properties of particles, and other quantities of interest. PDG IDs can have parent PDG IDs; for example, the parent PDG ID for a particle's mass will be the PDG ID of the particle itself. Top-level PDG IDs, such as those of particles, generally take the form of a letter followed by three digits. The PDG ID for a particle property takes the form of the parent PDG ID, a period, then some digits. A specific edition can be appended with a slash.

Unlike Monte Carlo IDs, which always identify specific states, a PDG ID can represent multiple states from a multiplet.

#### The `get` function

### Monte Carlo IDs 

Monte Carlo IDs (oftened called "PDG IDs" in the community) are used in MC generators and identify specific states. They are assigned algorithmically, with some historical special cases.

### Basic particle properties

Basic properties, particularly quantum numbers, are directly associated to particles.

### Summary table values 

Measured quantities, such as particle properties, are summarized by the PDG in *summary tables*, which list the PDG's evaluation (e.g. an average or fit) of the quantity based on measurements in the literature. The individual measurements are listed in *listings*. Currently, the API provides access to summary table values. Support for the listings will be added in the future.

A given PDG ID may have multiple summary table values, corresponding, for example, to different measurement techniques.

#### Parent PDG IDs 

A particle property listed in the summary tables will have the particle itself as its parent PDG ID.

#### Sort order

Quantities may have multiple summary table values, in which case the database specifies their ordering.

#### Flags

Additionally, when there are multiple summary table values for a quantity, they may use flags???? Or is this in pdgid/?????

#### "Best" summaries

When multiple values are available, the API can provide the "best" one, i.e. the first-sorted one with flags that match.

### Items and their mappings

Decays may be expressed in terms of "generic" particles: pions, leptons, etc. More generally, every "thing" in a decay is a *PDG item*. Some items correspond to specific particle states (MC IDs), while others correspond to other (specific) items, either mapping to one particular item (a *shortcut* or *alias*) or mapping to multiple (a *generic* item). The database defines this map between items. 

## Exercises

### Getting a particle

Particles are represented by the `PdgParticle` class. There are multiple ways to get a particle from the API. Depending on the method, the result can be a `PdgParticleList`, a generic `PdgParticle`, or a specific `PdgParticle`.

#### By name

A particle name can refer to a single particle or to a group of them. The function `get_particle_by_name` will return a `PdgParticle` if there's a unique match, and will raise an exception if not. The function `get_particles_by_name` always returns a list of `PdgParticles`.

In [2]:
api.get_particle_by_name('pi+')

PdgParticle('S008/2024', name='pi+')

In [3]:
api.get_particles_by_name('pi')

[PdgParticle('S008/2024', name='pi-'),
 PdgParticle('S009/2024', name='pi0'),
 PdgParticle('S008/2024', name='pi+')]

In [4]:
api.get_particles_by_name('N')

[PdgParticle('S017/2024', name='n'),
 PdgParticle('S016/2024', name='p'),
 PdgParticle('S017/2024', name='nbar'),
 PdgParticle('S016/2024', name='pbar')]

In [5]:
api.get_particles_by_name('Xi_c(2980)')

[PdgParticle('B130/2024', name='Xi_c(2970)0'),
 PdgParticle('B130/2024', name='Xibar_c(2970)0')]

In [6]:
api.get_particles_by_name('Xi_c(2970)')

[PdgParticle('B130/2024', name='Xi_c(2970)0'),
 PdgParticle('B130/2024', name='Xibar_c(2970)0')]

#### By MC ID

Since MC IDs are unambiguous, `get_particle_by_mcid` returns a `PdgParticle` directly.

In [7]:
api.get_particle_by_mcid(2212)

PdgParticle('S016/2024', name='p')

#### By PDG ID

The `get` function, given a particle's PDG ID, returns a list of all associated particles. Since `get` must return an instance of a `PdgData` subclass, the return type is a `PdgParticleList`, rather than a simple list of `PdgParticle`s (as returned by `get_particles_by_name`). 

In [8]:
api.get('S008/2024')

PdgParticleList('S008/2024')

However, a simple list is simple to get:

In [9]:
list(_)

[PdgParticle('S008/2024', name='pi+'), PdgParticle('S008/2024', name='pi-')]

A `PdgParticle` can also be instantiated directly.

In [10]:
PdgParticle(api, 'S009/2024')

PdgParticle('S009/2024', name='pi0')

When a PDG ID refers to more than one specific state, either `set_name` or `set_mcid` must be passed.

In [11]:
PdgParticle(api, 'S008/2024', set_name='pi-')

PdgParticle('S008/2024', name='pi-')

### Getting basic properties

Some properties (quantum numbers) are associated directly with a `PdgParticle`.

In [12]:
p = api.get_particle_by_name('pi0')
p.charge, p.quantum_I, p.quantum_G, p.quantum_J, p.quantum_P, p.quantum_C

(0.0, '1', '-', '0', '-', '+')

In [13]:
p = api.get_particle_by_name('pi+')
p.charge, p.quantum_I, p.quantum_G, p.quantum_J, p.quantum_P, p.quantum_C

(1.0, '1', '-', '0', '-', None)

In [14]:
p = api.get_particle_by_name('gamma')
p.charge, p.quantum_I, p.quantum_G, p.quantum_J, p.quantum_P, p.quantum_C

(0.0, '0,1', None, '1', '-', '-')

In [15]:
p = api.get_particle_by_name('eta')
p.charge, p.quantum_I, p.quantum_G, p.quantum_J, p.quantum_P, p.quantum_C

(0.0, '0', '+', '0', '-', '+')

### Getting masses, widths, lifetimes

Masses, widths, and lifetimes are some of the particle properties that have their own PDG IDs. In some cases there may be multiple masses (or widths or lifetimes) for a given particle, corresponding, for example, to different measurement techniques.

For convenience, the `mass`, `width`, and `lifetime` properties of a `PdgParticle` can be used to get the "best" summary value. Widths and lifetimes are automatically interconverted when necessary. The full sets of summary values, as `PdgProperty` (subclass) objects, via the `masses`, `widths`, and `lifetimes` methods.

In [16]:
p = api.get_particle_by_name('pi+')

(p.mass, list(p.masses())), (p.width, list(p.widths())), (p.lifetime, list(p.lifetimes()))

((0.13957039098368132, [PdgMass('S008M/2024')]),
 (2.5283166082854054e-17, []),
 (2.603313199949126e-08, [PdgLifetime('S008T/2024')]))

In [17]:
p = api.get_particle_by_name('pi0')

(p.mass, list(p.masses())), (p.width, list(p.widths())), (p.lifetime, list(p.lifetimes()))

((0.1349768277676847, [PdgMass('S009M/2024')]),
 (7.811987971364424e-09, []),
 (8.425512205250367e-17, [PdgLifetime('S009T/2024')]))

In [18]:
p = api.get_particle_by_name('gamma')

(p.mass, list(p.masses())), (p.width, list(p.widths())), (p.lifetime, list(p.lifetimes()))

((1e-27, [PdgMass('S000M/2024')]), (0.0, []), (inf, []))

In [19]:
p = api.get_particle_by_name('t')

(p.mass, list(p.masses())), (p.width, list(p.widths())), (p.lifetime, list(p.lifetimes()))

((172.5746930968864,
  [PdgMass('Q007TP/2024'), PdgMass('Q007TP2/2024'), PdgMass('Q007TP4/2024')]),
 (1.424101758196898, [PdgWidth('Q007W/2024')]),
 (4.621860735804221e-25, []))

### Getting limits on things

In [20]:
p = api.get_particle_by_name('gamma')
p.mass

1e-27

In [21]:
m = list(p.masses())[0]

In [22]:
m

PdgMass('S000M/2024')

In [23]:
m.data_flags

'D'

In [24]:
m.data_type

'M'

In [25]:
m.summary_values()

[{'id': 1020000,
  'pdgid_id': 1,
  'pdgid': 'S000M',
  'edition': '2024',
  'value_type': 'L',
  'in_summary_table': True,
  'confidence_level': None,
  'limit_type': 'U',
  'comment': None,
  'value': 1e-18,
  'error_positive': 0.0,
  'error_negative': 0.0,
  'scale_factor': None,
  'unit_text': 'eV',
  'display_value_text': '<1E-18',
  'display_power_of_ten': 0,
  'display_in_percent': False,
  'sort': 1,
  'description': 'gamma MASS'}]

In [26]:
m.best_summary()

{'id': 1020000,
 'pdgid_id': 1,
 'pdgid': 'S000M',
 'edition': '2024',
 'value_type': 'L',
 'in_summary_table': True,
 'confidence_level': None,
 'limit_type': 'U',
 'comment': None,
 'value': 1e-18,
 'error_positive': 0.0,
 'error_negative': 0.0,
 'scale_factor': None,
 'unit_text': 'eV',
 'display_value_text': '<1E-18',
 'display_power_of_ten': 0,
 'display_in_percent': False,
 'sort': 1,
 'description': 'gamma MASS'}

### Getting a particle's decays

In [27]:
p = api.get_particle_by_name('pi+')

In [28]:
list(p.inclusive_branching_fractions())

[]

In [29]:
list(p.exclusive_branching_fractions())

[PdgBranchingFraction('S008.1/2024'),
 PdgBranchingFraction('S008.2/2024'),
 PdgBranchingFraction('S008.4/2024'),
 PdgBranchingFraction('S008.6/2024'),
 PdgBranchingFraction('S008.11/2024'),
 PdgBranchingFraction('S008.10/2024'),
 PdgBranchingFraction('S008.7/2024'),
 PdgBranchingFraction('S008.8/2024'),
 PdgBranchingFraction('S008.9/2024')]

In [30]:
decay = _[0]

In [31]:
decay

PdgBranchingFraction('S008.1/2024')

In [32]:
decay.description

'pi+ --> mu+ nu_mu'

In [33]:
decay.summary_values()

[{'id': 1021202,
  'pdgid_id': 1306,
  'pdgid': 'S008.1',
  'edition': '2024',
  'value_type': 'V',
  'in_summary_table': True,
  'confidence_level': None,
  'limit_type': None,
  'comment': None,
  'value': 99.9877,
  'error_positive': 4e-07,
  'error_negative': 4e-07,
  'scale_factor': None,
  'unit_text': '',
  'display_value_text': '(99.98770+-0.00004)%',
  'display_power_of_ten': 0,
  'display_in_percent': False,
  'sort': 1,
  'description': 'pi+ --> mu+ nu_mu'}]

In [34]:
decay.value

99.9877

### Iterating over decay products

In [35]:
products = decay.decay_products

Shouldn't that be a function that returns an iterator?

In [36]:
prod = products[0]

prod.item, prod.multiplier, prod.subdecay

(PdgItem("mu+"), 1, None)

In [37]:
prod.item.particle

PdgParticle('S004/2024', name='mu+')

In [38]:
prod = products[1]

prod.item, prod.multiplier, prod.subdecay

(PdgItem("nu_mu"), 1, None)

In [39]:
prod.item.particle

PdgParticle('S002/2024', name='nu_mu')

Now show one where there's a generic item where `.particle` fails

### Getting arbitrary properties

#### Particle-associated properties

In [40]:
v = api.get('Q007TP4')
v.description, v.value, v.units

('t-Quark Pole Mass from Cross-Section Measurements', 172.3523553288312, 'GeV')

In [41]:
v.summary_values()

[{'id': 1021134,
  'pdgid_id': 1175,
  'pdgid': 'Q007TP4',
  'edition': '2024',
  'value_type': 'AC',
  'in_summary_table': True,
  'confidence_level': None,
  'limit_type': None,
  'comment': None,
  'value': 172.3523553288312,
  'error_positive': 0.6752618288606228,
  'error_negative': 0.6432013333753764,
  'scale_factor': 1.0,
  'unit_text': 'GeV',
  'display_value_text': '172.4+-0.7',
  'display_power_of_ten': 0,
  'display_in_percent': False,
  'sort': 1,
  'description': 't-Quark Pole Mass from Cross-Section Measurements'}]

#### Particle-independent properties

##### Neutrino properties

These live under `S066` and `S067`.

In [42]:
v = api.get('S067P23')
v.description, v.value

('sin**2(theta(23))', 0.5529894603427739)

In [43]:
v.summary_values()

[{'id': 1021101,
  'pdgid_id': 1136,
  'pdgid': 'S067P23',
  'edition': '2024',
  'value_type': 'FC',
  'in_summary_table': True,
  'confidence_level': None,
  'limit_type': None,
  'comment': 'Assuming inverted mass ordering',
  'value': 0.5529894603427739,
  'error_positive': 0.01557018799120562,
  'error_negative': 0.02376487951333883,
  'scale_factor': 1.053519,
  'unit_text': '',
  'display_value_text': '0.553+0.016-0.024',
  'display_power_of_ten': 0,
  'display_in_percent': False,
  'sort': 1,
  'description': 'sin**2(theta(23))'},
 {'id': 1021102,
  'pdgid_id': 1136,
  'pdgid': 'S067P23',
  'edition': '2024',
  'value_type': 'FC',
  'in_summary_table': True,
  'confidence_level': None,
  'limit_type': None,
  'comment': 'Assuming normal mass ordering',
  'value': 0.5579671857245437,
  'error_positive': 0.01540986825925941,
  'error_negative': 0.02051779341399617,
  'scale_factor': 1.0,
  'unit_text': '',
  'display_value_text': '0.558+0.015-0.021',
  'display_power_of_ten': 0,


In [44]:
prop = api.get('S067')

##### CKM elements

### Getting all of the things

#### Particles

#### Decays

## Fancier exercises

### Getting all particles with nonzero strangeness

### Printing all decays that produce a $J/\psi$

### Plotting masses of all decay products of $\Upsilon(4S)$

### Plotting precision of \<some_measurement\> over time

### Getting all isospin partners of \<some_particle\>

## Direct access using SQLAlchemy

## Metadata and documentation tables